# This is a trial for onnx to convert tensorflow model

In [53]:
import tensorflow as tf
from tensorflow import keras
import tf2onnx
import numpy as np
import onnxruntime as ort

In [54]:
tf.__version__

'1.14.0'

In [55]:
tf2onnx.__version__

'1.5.3'

In [56]:
ort.__version__

'0.4.0'

# A simple MNIST handwritten digit classification using tensorFlow

In [57]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [58]:
import matplotlib.pyplot as plt

In [59]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 7s 115us/sample - loss: 0.2204 - acc: 0.9352
Epoch 2/5
60000/60000 [==============================] - 7s 110us/sample - loss: 0.0978 - acc: 0.9706
Epoch 3/5
60000/60000 [==============================] - 7s 114us/sample - loss: 0.0690 - acc: 0.9789
Epoch 4/5
60000/60000 [==============================] - 7s 114us/sample - loss: 0.0526 - acc: 0.9829
Epoch 5/5
60000/60000 [==============================] - 7s 114us/sample - loss: 0.0433 - acc: 0.9861


In [62]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 65us/sample - loss: 0.0719 - acc: 0.9802


[0.07194251735692378, 0.9802]

In [21]:
prediction = model.predict(x_test)

In [22]:
print(prediction[0])

[1.2286372e-06 9.6285795e-08 1.4894309e-06 2.1995955e-04 7.7952559e-13
 7.3951538e-09 2.7167164e-11 9.9977666e-01 9.4978347e-08 4.0528519e-07]


In [23]:
print(y_test[0])

7


In [63]:
print("the digit is classified as \"%s\" in prediction"%np.argmax(prediction[0]))

the digit is classified as "7" in prediction


In [64]:
saved_model_path = "./saved_mnist"

In [65]:
keras.experimental.export_saved_model(model, saved_model_path)

W0903 23:56:07.450877 139795298592512 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0903 23:56:07.452551 139795298592512 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0903 23:56:07.453497 139795298592512 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0903 23:56:07.454456 139795298592512 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0903 23:56:07.455389 139795298592512 util.py:244] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.kernel
W0903 23:56:07.456325 139795298592512 util.py:244] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.bias
W0903 23:56:07.457236 139795298592512 util.py:244] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0903 23:56:07.458156 139795298592512 util.py:244] Unresolved object in checkpoint: (root).optimizer's state

# Converting .pb model to .onnx and verify the prediction using ONNXRruntime

In [27]:
#python -m tf2onnx.convert --saved-model ./saved_mnist/ --output ./saved_mnist/saved_model.onnx

In [66]:
import numpy as np
import onnxruntime as ort

In [67]:
sess_ort = ort.InferenceSession("./saved_mnist/saved_model.onnx")

In [68]:
input_name = sess_ort.get_inputs()[0].name
print("input name", input_name)
input_shape = sess_ort.get_inputs()[0].shape
print("input shape", input_shape)
input_type = sess_ort.get_inputs()[0].type
print("input type", input_type)

input name flatten_1_input:0
input shape [None, 28, 28]
input type tensor(float)


In [69]:
output_name = sess_ort.get_outputs()[0].name
print("output_name", output_name)
output_shape = sess_ort.get_outputs()[0].shape
print("output shape", output_shape)
output_type = sess_ort.get_outputs()[0].type
print("output type", output_type)

output_name dense_3/Softmax:0
output shape [None, 10]
output type tensor(float)


# Using a test sample for validation

In [77]:
img = x_test[0]
img = img.astype(np.float32)

In [78]:
img=img.reshape(1,28,28)
img.shape

(1, 28, 28)

In [79]:
res = sess_ort.run([output_name], {input_name: img})

In [80]:
print("the digit is classified as \"%s\" in ONNXRruntime"%np.argmax(res))

the digit is classified as "7" in ONNXRruntime
